# Derivative of traits_i with respect to traits_i

Here I will be testing my solution for
$\frac{ \partial \mathbf{\hat{V}_i} }{ \partial \mathbf{V_i} }$
(see below)
by calculating the Jacobian using the `theano` package
and comparing those results to my solution.




## Importing packages and setting options

In [2]:
%env OMP_NUM_THREADS=4
%env THEANO_FLAGS='openmp=True'
import sympy
import theano
theano.config.cxx = ""
import theano.tensor as T
import numpy as np
import pandas as pd
from tqdm import tqdm
import math
pd.options.display.max_columns = 10

env: OMP_NUM_THREADS=4
env: THEANO_FLAGS='openmp=True'


## Equations

__Notes:__

- ${}^\text{T}$ represents transpose.
- Elements in __bold__ are matrices
- Multiplication between matrices is always matrix multiplication, not
  element-wise
  

The equations for (1) traits for species $i$ at time $t+1$ ($\mathbf{V}_{i,t+1}$)
and (2) the partial derivative of species $i$ traits at time $t+1$ with respect
to species $i$ traits at time  $t$ are as follows:

\begin{align}
\mathbf{V}_{i,t+1} &= \mathbf{V}_{i,t} + 2 ~ \sigma_i^2
    \left[
        \alpha_0 ~ \mathbf{\Omega}_{i,t} ~
            \textrm{e}^{-\mathbf{V}_{i,t} \mathbf{V}_{i,t}^\textrm{T}} ~ \mathbf{V}_{i,t}
        - f ~ \mathbf{V}_{i,t} \mathbf{C}
    \right] \\
    \frac{ \partial \mathbf{V}_{i,t+1} }{ \partial \mathbf{V}_{i,t} } &= \mathbf{I} + 2 ~ \sigma_i^2 ~
        \left[
            \alpha_0 ~ \mathbf{\Omega}_{i,t} ~ \textrm{e}^{ - \mathbf{V}_{i,t} \mathbf{V}_{i,t}^{\textrm{T}} }
            \left(
                -2 ~ \mathbf{V}_{i,t}^{\textrm{T}} \mathbf{V}_{i,t} + \mathbf{I}
            \right) -
            f \mathbf{C}
        \right] \\
    \mathbf{\Omega}_{i,t} &\equiv N_{i,t} +
            \sum_{j \ne i}^{n}{ N_{j,t} \textrm{e}^{ - \mathbf{V}_{j,t} \mathbf{D} \mathbf{V}_{j,t}^{\textrm{T}} } }
\end{align}


## Read CSV of simulated datasets

In [3]:
sims = pd.read_csv("simulated_data.csv")
sims.head()

,V1,V2,V3,V4,V5,...,f,a0,eta,r0,d
0,4.945110,2.869199,6.747126,6.142522,5.629532,...,0.068890,0.112113,-0.331150,1.422746,-0.091228
1,0.718846,1.220364,0.815571,0.868633,0.838021,...,0.309021,0.057579,0.094811,1.237047,0.003429
2,3.369285,1.912974,3.131174,0.046303,1.416252,...,0.118318,0.401410,-0.036977,1.746024,0.012160
3,0.373669,0.283873,0.237735,0.053632,0.062281,...,0.497286,0.499730,0.117188,0.669199,0.081612
4,3.562637,1.635016,5.724176,4.953962,1.060083,...,0.042638,0.307171,-0.467453,0.952351,0.051834


## Functions to compare methods

In [4]:
def automatic(i, N, V, O, C, f, a0, s2):
    """Automatic differentiation using theano pkg"""
    Vi = T.dvector('Vi')
    Vhat = Vi + 2 * s2 * (
        ( a0 * O * T.exp(-1 * T.dot(Vi, Vi.T)) * Vi) - 
        ( f * T.dot(Vi, C) )
    )
    J, updates = theano.scan(lambda i, Vhat, Vi : T.grad(Vhat[i], Vi), 
                         sequences=T.arange(Vhat.shape[0]), non_sequences=[Vhat, Vi])
    num_fun = theano.function([Vi], J, updates=updates)
    out_array = num_fun(V[i,:]).T
    return out_array

In [5]:
def symbolic(i, V, O, C, f, a0, s2):
    """Symbolic differentiation using math"""
    q = V.shape[1]
    I = np.identity(q)
    Vi = V[i,:]
    Vi = Vi.reshape((1, q))
    dVhat = I + 2 * s2 * (
        ( a0 * O * np.exp(-1 * Vi @ Vi.T)[0,0] * (I - 2 * Vi.T @ Vi) ) -
        (f * C)
    )
    return dVhat

In [6]:
def compare_methods(sim_i, s2 = 0.01, abs = False):
    """Compare answers from symbolic and automatic methods"""
    
    # Fill info from data frame:
    N = sims.loc[sim_i, [x.startswith("N") for x in sims.columns]].values
    V = sims.loc[sim_i, [x.startswith("V") for x in sims.columns]].values
    n, q = (N.size, int(V.size / N.size))
    V = V.reshape((n, q), order = 'F')
    f = sims.loc[sim_i,"f"]
    a0 = sims.loc[sim_i,"a0"]
    eta = sims.loc[sim_i,"eta"]
    d = sims.loc[sim_i,"d"]
    D = np.zeros((q, q))
    np.fill_diagonal(D, d)
    C = np.zeros((q, q)) + eta
    np.fill_diagonal(C,1.0)
    
    # Create output array:
    diffs = np.empty((n, 4))
    diffs[:,0] = sim_i
    
    # Fill output array:
    for i in range(0, n):
        O = N[i] + np.sum([np.exp(-1 * np.dot(np.dot(V[j,:], D), V[j,:].T)) * N[j] 
            for j in range(0, N.size) if j != i])
        auto = automatic(i, N, V, O, C, f, a0, s2)
        sym = symbolic(i, V, O, C, f, a0, s2)
        if abs:
            diff = auto - sym
        else:
            diff = (auto - sym) / sym
        diff = diff.flatten()
        diffs[i, 1] = i
        diffs[i, 2] = diff.min()
        diffs[i, 3] = diff.max()
    
    return diffs

In [9]:
sim_i = 0
s2 = 0.01
abs = False
# Fill info from data frame:
N = sims.loc[sim_i, [x.startswith("N") for x in sims.columns]].values
V = sims.loc[sim_i, [x.startswith("V") for x in sims.columns]].values
n, q = (N.size, int(V.size / N.size))
V = V.reshape((n, q), order = 'F')
f = sims.loc[sim_i,"f"]
a0 = sims.loc[sim_i,"a0"]
eta = sims.loc[sim_i,"eta"]
d = sims.loc[sim_i,"d"]
D = np.zeros((q, q))
np.fill_diagonal(D, d)
C = np.zeros((q, q)) + eta
np.fill_diagonal(C,1.0)

# Create output array:
diffs = np.empty((n, 4))
diffs[:,0] = sim_i

# Fill output array:
# for i in range(0, n):
i = 0
O = N[i] + np.sum([np.exp(-1 * np.dot(np.dot(V[j,:], D), V[j,:].T)) * N[j] 
    for j in range(0, N.size) if j != i])


print("\n i")
print(i)
print("\n N")
print(N)
print("\n V")
print(V)
print("\n O")
print(O)
print("\n C")
print(C)
print("\n f")
print(f)
print("\n a0")
print(a0)
print("\n s2")
print(s2)

auto = automatic(i, N, V, O, C, f, a0, s2)
sym = symbolic(i, V, O, C, f, a0, s2)

print("\n\n auto")
print(auto)
print("\n sym")
print(sym)




 i
0

 N
[ 7.18015044  5.02769558 14.05186271  7.84885382]

 V
[[4.94511003 5.62953205 4.26294319]
 [2.86919893 1.8334118  1.88296186]
 [6.74712574 3.67306824 2.71730932]
 [6.14252194 3.10081916 2.42625001]]

 O
7040.411110648025

 C
[[ 1.         -0.33114977 -0.33114977]
 [-0.33114977  1.         -0.33114977]
 [-0.33114977 -0.33114977  1.        ]]

 f
0.06888985494151711

 a0
0.11211295961402357

 s2
0.01


 auto
[[9.98622203e-01 4.56257198e-04 4.56257198e-04]
 [4.56257198e-04 9.98622203e-01 4.56257198e-04]
 [4.56257198e-04 4.56257198e-04 9.98622203e-01]]

 sym
[[9.98622203e-01 4.56257198e-04 4.56257198e-04]
 [4.56257198e-04 9.98622203e-01 4.56257198e-04]
 [4.56257198e-04 4.56257198e-04 9.98622203e-01]]


### Example of using `compare_methods`:

In [10]:
diffs = compare_methods(0)
# Worst case examples:
print(diffs[:,2].min())
print(diffs[:,3].max())

-2.910430995840879e-16
1.188148020236418e-16


## Comparing methods

This takes ~2 minutes.

In [11]:
n_per_rep = 4
diffs = np.empty((int(n_per_rep * 100), 4))

In [12]:
for rep in tqdm(range(100)):
    diffs_r = compare_methods(rep)
    diffs[(rep * n_per_rep):((rep+1) * n_per_rep),:] = diffs_r

100%|██████████| 100/100 [02:47<00:00,  1.68s/it]


## The results
They appear to be extremely similar, enough so that I feel comfortable with my symbolic version.

In [13]:
print(diffs[:,2].min())
print(diffs[:,3].max())

-2.095476089723135e-14
1.7507984967188207e-14


## Write output to file

To make sure the R version works, too, I'm writing to a CSV file the output from the symbolic version on the 100 datasets.

In [14]:
n = np.sum([x.startswith("N") for x in sims.columns])
q = int(np.sum([x.startswith("V") for x in sims.columns]) / n)
s2 = 0.01
# Output array
results = np.zeros((100, n * q * q))

for sim_i in range(100):
    
    # Fill info from data frame:
    N = sims.loc[sim_i, [x.startswith("N") for x in sims.columns]].values
    V = sims.loc[sim_i, [x.startswith("V") for x in sims.columns]].values
    V = V.reshape((n, q), order = 'F')
    f = sims.loc[sim_i,"f"]
    a0 = sims.loc[sim_i,"a0"]
    eta = sims.loc[sim_i,"eta"]
    d = sims.loc[sim_i,"d"]
    C = np.zeros((q, q)) + eta
    np.fill_diagonal(C,1.0)

    # Fill output array:
    for i in range(0, n):
        O = N[i] + np.sum([np.exp(-d * np.dot(V[j,:], V[j,:].T)) * N[j] 
            for j in range(0, N.size) if j != i])
        sym = symbolic(i, V, O, C, f, a0, s2)
        results[sim_i, (i*q*q):((i+1)*q*q)] = sym.flatten()

# Make sure first and last aren't zeros:
results[[0, 99], :]

array([[ 9.98622203e-01,  4.56257198e-04,  4.56257198e-04,
         4.56257198e-04,  9.98622203e-01,  4.56257198e-04,
         4.56257198e-04,  4.56257198e-04,  9.98622203e-01,
         9.98499122e-01,  3.72522892e-04,  3.70259877e-04,
         3.72522892e-04,  9.98576656e-01,  4.01305099e-04,
         3.70259877e-04,  4.01305099e-04,  9.98573725e-01,
         9.98622203e-01,  4.56257198e-04,  4.56257198e-04,
         4.56257198e-04,  9.98622203e-01,  4.56257198e-04,
         4.56257198e-04,  4.56257198e-04,  9.98622203e-01,
         9.98622203e-01,  4.56257198e-04,  4.56257198e-04,
         4.56257198e-04,  9.98622203e-01,  4.56257198e-04,
         4.56257198e-04,  4.56257198e-04,  9.98622203e-01],
       [ 9.96860545e-01,  6.14604274e-04,  6.65366900e-04,
         6.14604274e-04,  9.96794653e-01,  6.58923055e-04,
         6.65366900e-04,  6.58923055e-04,  9.96893697e-01,
         1.03264893e+00, -8.22446029e-02, -1.60952904e-02,
        -8.22446029e-02,  9.78892211e-01, -2.23903463e-

In [15]:
np.savetxt('results/dVi_dVi.csv', results, delimiter=',')